<a href="https://colab.research.google.com/github/matheusvazdata/desafio-3-trilhas-2b/blob/main/etl_desafio_3_trilhas_2b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ✅ Objetivo
Transformar uma simples planilha em um pipeline bem estruturado, limpo e confiável até a visualização no Looker Studio, com potencial para escalar futuramente.

## 🧱 Arquitetura Moderna Recomendada (Camadas e Ferramentas)

```
[Google Sheets] (raw)
      ↓
[Google Colab ou Cloud Functions com Python]
      ↓
[Cloud Storage ou BigQuery] ← (zona trusted / staging)
      ↓
[Looker Studio] (visualização)
```

### 🔄 ETL no Colab (boas práticas)
1. Extract – Leitura direta do Google Sheets
Use a API do Google Sheets ou o gspread + oauth2client.

In [53]:
# Importação das bibliotecas necessárias
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import numpy as np

In [54]:
# Autenticação
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('/content/minha-chave.json', scope)
client = gspread.authorize(creds)

# Carregar a planilha
sheet = client.open_by_url('https://docs.google.com/spreadsheets/d/17hAlR-OjU2mJ6mlLhIPPRWVD02Mn_js21Zx4HuYo2b8')
worksheet = sheet.sheet1
dados = pd.DataFrame(worksheet.get_all_records())

2. Transform – Limpeza e enriquecimento
- 2.1. 🧪 Testes automáticos

In [55]:
def validar_dados(dados):
    assert not dados.isnull().all().any(), '❌ Colunas totalmente nulas.'

    colunas_constantes = [col for col in dados.columns if dados[col].nunique(dropna=True) == 1]
    assert len(colunas_constantes) == 0, f'⚠️ Colunas constantes: {colunas_constantes}'

    assert not dados.duplicated().any(), '❌ Linhas duplicadas.'

    colunas_esperadas = ['idade', 'trilha', 'gênero', 'município']
    for col in colunas_esperadas:
        assert col in dados.columns, f'❌ Coluna ausente: {col}'

    assert dados['trilha'].notnull().all(), '❌ "trilha" com valores nulos.'
    assert dados['idade'].notnull().all(), '❌ "idade" com valores nulos.'

    assert pd.api.types.is_numeric_dtype(dados['idade']), '❌ "idade" não é numérica.'

    print('✅ Validação de dados concluída com sucesso!')

# Chamada:
validar_dados(dados)

AssertionError: ⚠️ Colunas constantes: ['Nome Completo']

In [56]:
!pip install unidecode

In [57]:
# 🔹 Renomear colunas para snake_case com limpeza extra
from unidecode import unidecode

# 🔹 Renomear colunas com limpeza completa
dados.columns = [
    unidecode(col)                      # remove acentos
    .strip()                            # remove espaços nas extremidades
    .lower()                            # caixa baixa
    .replace(' ', '_')                 # espaços → _
    .replace('/', '_')                 # barras → _
    .replace('-', '_')                 # hífens → _
    .replace(':', '')                  # remove dois-pontos
    .replace('__', '_')                # evita underscore duplo
    .strip('_')                        # remove underscores nas pontas
    for col in dados.columns
]

# 🔹 Remover colunas constantes antes de continuar
colunas_constantes = [col for col in dados.columns if dados[col].nunique(dropna=True) == 1]
dados = dados.drop(columns=colunas_constantes)

# 🔹 Criar faixa etária
dados['faixa_etaria'] = pd.cut(dados['idade'], bins=[17, 20, 25, 30, 40], labels=['18-20', '21-25', '26-30', '31-40'])

# 🔹 Criar colunas simplificadas
dados['trilha_simplificada'] = np.where(
    dados['trilha'].str.contains('Programação', na=False),
    dados['trilha'].str.extract(r'\(([^)]+)\)', expand=False).fillna('Jogos'),
    dados['trilha']
)
dados['escola_publica'] = dados['escola_de_origem'].str.lower().eq('pública')

# 🔹 Corrigir campos booleanos
dados['tem_acesso'] = dados['acesso_a_net_e_ao_pc'].map({'Sim': True, 'Não': False})

# 🔹 Normalizar gênero
dados['genero'] = dados['genero'].fillna('').str.strip().str.title()

# 🔹 Substituir qualquer valor que não seja "Masculino" ou "Feminino"
dados['genero'] = dados['genero'].apply(lambda x: x if x in ['Masculino', 'Feminino'] else 'Não informado')

In [58]:
dados.head()

,carimbo_de_data_hora,idade,municipio,trilha,genero,ocupacao_atual_emprego,cor_ou_raca,grau_de_escolaridade,participou_do_trilhas_anterior,acesso_a_net_e_ao_pc,faixa_de_renda_familiar,motivo_da_participacao,escola_de_origem,faixa_etaria,trilha_simplificada,escola_publica,tem_acesso
0,2023/12/15 13:18:17,24,São Luís,Programação (Front-end),Feminino,Nunca Trabalhei,Parda,Graduação Incompleta (ou cursando),Não,Sim,Até R$ 1.302,Ter formação em uma das Trilhas,Pública,21-25,Front-end,True,True
1,2023/12/15 14:11:49,21,São Luís,Programação (Back-end),Masculino,Desempregado,Parda,Graduação Incompleta (ou cursando),Não,Sim,Até R$ 1.302,Conseguir um emprego,Pública,21-25,Back-end,True,True
2,2023/12/15 14:16:07,24,São Luís,Programação (Front-end),Masculino,Desempregado,Parda,Graduação Incompleta (ou cursando),Não,Sim,Entre R$ 1.303 e R$ 2.605,Mudança de carreira,Pública,21-25,Front-end,True,True
3,2023/12/15 14:24:20,22,São Luís,Programação (Back-end),Masculino,"Não estou empregado, mas focando apenas nos es...",Parda,Graduação Incompleta (ou cursando),Não,Sim,Entre R$ 1.303 e R$ 2.605,Ter formação em uma das Trilhas,Pública,21-25,Back-end,True,True
4,2023/12/15 14:35:51,23,São Luís,Design e Experiência,Feminino,A busca de emprego,Parda,Graduação Incompleta (ou cursando),Não,Sim,Até R$ 1.302,Conseguir um emprego,Pública,21-25,Design e Experiência,True,True


In [59]:
dados.trilha_simplificada.unique()

array(['Front-end', 'Back-end', 'Design e Experiência', 'Jogos',
       'Ciência de Dados'], dtype=object)

3. Load – Armazenar em formato adequado

In [60]:
!pip install --upgrade google-cloud-bigquery

In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account

# Caminho do JSON de credenciais
caminho_credenciais = '/content/minha-chave.json'

# Criar o client autenticado
credenciais = service_account.Credentials.from_service_account_file(caminho_credenciais)
client = bigquery.Client(credentials=credenciais, project=credenciais.project_id)

# Defina o ID da tabela (use seu projeto e dataset)
tabela_id = 'meu-id-do-projeto.meu-dataset.minha-tabela'

from google.cloud import bigquery

# Configurar o job para criar a tabela se não existir
job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")

job = client.load_table_from_dataframe(dados, tabela_id, job_config=job_config)
job.result()

print(f'✅ Dados enviados para: {tabela_id}')